<a href="https://colab.research.google.com/github/Othavyol/Calculadora-em-python/blob/main/Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import copy
import heapq
from collections import deque
import time

# Estado objetivo
goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

# Movimentos possíveis
moves = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

def find_zero(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j

def get_neighbors(state):
    neighbors = []
    x, y = find_zero(state)
    for dx, dy in moves.values():
        nx, ny = x + dx, y + dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            new_state = copy.deepcopy(state)
            new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
            neighbors.append(new_state)
    return neighbors

def is_goal(state):
    return state == goal_state

def heuristic(state):
    # Número de peças fora do lugar
    count = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0 and state[i][j] != goal_state[i][j]:
                count += 1
    return count

# ---------------- BFS ----------------
def bfs(start):
    start_time = time.time()
    queue = deque([start])
    visited = set()
    visited.add(str(start))
    parent = {str(start): None}
    nodes_expanded = 0

    while queue:
        state = queue.popleft()
        nodes_expanded += 1

        if is_goal(state):
            path = []
            while state:
                path.append(state)
                state = parent[str(state)]
            end_time = time.time()
            return path[::-1], nodes_expanded, len(path)-1, end_time - start_time

        for neighbor in get_neighbors(state):
            neighbor_str = str(neighbor)
            if neighbor_str not in visited:
                visited.add(neighbor_str)
                parent[neighbor_str] = state
                queue.append(neighbor)

# ---------------- DFS ----------------
def dfs(start):
    start_time = time.time()
    stack = [start]
    visited = set()
    parent = {str(start): None}
    nodes_expanded = 0

    while stack:
        state = stack.pop()
        nodes_expanded += 1

        if is_goal(state):
            path = []
            while state:
                path.append(state)
                state = parent[str(state)]
            end_time = time.time()
            return path[::-1], nodes_expanded, len(path)-1, end_time - start_time

        visited.add(str(state))
        for neighbor in get_neighbors(state):
            neighbor_str = str(neighbor)
            if neighbor_str not in visited:
                parent[neighbor_str] = state
                stack.append(neighbor)

# ---------------- A* ----------------
def a_star(start):
    start_time = time.time()
    heap = []
    g_score = {str(start): 0}
    heapq.heappush(heap, (heuristic(start), 0, start))
    parent = {str(start): None}
    visited = set()
    nodes_expanded = 0

    while heap:
        f, g, state = heapq.heappop(heap)
        nodes_expanded += 1

        if is_goal(state):
            path = []
            while state:
                path.append(state)
                state = parent[str(state)]
            end_time = time.time()
            return path[::-1], nodes_expanded, len(path)-1, end_time - start_time

        visited.add(str(state))

        for neighbor in get_neighbors(state):
            neighbor_str = str(neighbor)
            tentative_g = g + 1
            if neighbor_str not in visited or tentative_g < g_score.get(neighbor_str, float('inf')):
                g_score[neighbor_str] = tentative_g
                f_score = tentative_g + heuristic(neighbor)
                heapq.heappush(heap, (f_score, tentative_g, neighbor))
                parent[neighbor_str] = state

# ---------------- Teste ----------------
start_state = [
    [1, 2, 3],
    [4, 0, 6],
    [7, 5, 8]
]

def print_results(name, result):
    path, nodes, depth, elapsed = result
    print(f"\n{name}:")
    print(f"Nós expandidos: {nodes}")
    print(f"Profundidade da solução: {depth}")
    print(f"Tempo de execução: {elapsed:.6f}s")

print_results("BFS", bfs(start_state))
print_results("DFS", dfs(start_state))
print_results("A*", a_star(start_state))


BFS:
Nós expandidos: 9
Profundidade da solução: 2
Tempo de execução: 0.000284s

DFS:
Nós expandidos: 441
Profundidade da solução: 434
Tempo de execução: 0.013758s

A*:
Nós expandidos: 3
Profundidade da solução: 2
Tempo de execução: 0.000116s
